In [3]:
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split


movie_data_path = Path("movie_data.csv")
movie_data_df = pd.read_csv(movie_data_path)
movie_data_df.head()



,Unnamed: 0,adult,backdrop_path,belongs_to_collection,budget,genres,genres_0_name,genres_1_name,genres_2_name,genres_3_name,...,spoken_languages_7_name,spoken_languages_8_english_name,spoken_languages_8_iso_639_1,spoken_languages_8_name,status,tagline,title,video,vote_average,vote_count
0,0,False,/417tYZ4XUyJrtyZXj7HpvWf1E8f.jpg,NaN,78000000,NaN,Animation,Science Fiction,Family,NaN,...,NaN,NaN,NaN,NaN,Released,Discover your true nature.,The Wild Robot,False,8.666,1403
1,1,False,/9SSEUrSqhljBMzRe4aBTh17rUaC.jpg,NaN,80000000,NaN,Science Fiction,Horror,NaN,NaN,...,NaN,NaN,NaN,NaN,Released,"In space, no one can hear you.",Alien: Romulus,False,7.300,1834
2,2,False,/7h6TqPB3ESmjuVbxCxAeB1c9OB1.jpg,NaN,17500000,NaN,Horror,Drama,Science Fiction,NaN,...,NaN,NaN,NaN,NaN,Released,"If you follow the instructions, what could go ...",The Substance,False,7.300,839
3,3,False,/3sh2UA2Q2l7fihgoj1LhFFPTlIM.jpg,NaN,110000000,NaN,Action,Science Fiction,Adventure,NaN,...,NaN,NaN,NaN,NaN,Released,'Til death do they part.,Venom: The Last Dance,False,7.100,32
4,4,False,/yDHYTfA3R0jFYba16jBB1ef8oIt.jpg,NaN,200000000,NaN,Action,Comedy,Science Fiction,NaN,...,NaN,NaN,NaN,NaN,Released,Come together.,Deadpool & Wolverine,False,7.700,4764


In [4]:
movie_data_train, movie_data_test = train_test_split(movie_data_df, test_size=0.2, random_state=42)
movie_data_train.to_csv("movie_data_train.csv", index=False)
movie_data_test.to_csv("movie_data_test.csv", index=False)